# Dynamic Scene

## Setup Sim

### basic setup

In [12]:
import carla
import math
import random
import time
import queue
import numpy as np
import cv2
import matplotlib.pyplot as plt

client = carla.Client('localhost', 2000)
world  = client.get_world()
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

#### crossings location (default map)

In [13]:
spectator = world.get_spectator() 
#Intersection1 (checkerboard)
inter1tv = carla.Transform(
    carla.Location(
        x=-45,
        y=20,
        z=60
    ),
    carla.Rotation(
        pitch=-90,
        yaw=0,
        roll=0
    )
)

inter1pv1 = carla.Transform(
    carla.Location(
        x=-60,
        y=40,
        z=7
    ),
    carla.Rotation(
        pitch=-30,
        yaw=-60,
        roll=0
    )
)

inter1pv2 = carla.Transform(
    carla.Location(
        x=-60,
        y=20,
        z=7.3
    ),
    carla.Rotation(
        pitch=-28,
        yaw=33.5,
        roll=0
    )
)

inter1pv3 = carla.Transform(
    carla.Location(
        x = -63.7,
        y = 7.9,
        z = 4.5
    ),
    carla.Rotation(
        pitch = -32,
        yaw = 41.8,
        roll = 0
    )
)

inter1pv4 = carla.Transform(
    carla.Location(
        x = -35.9,
        y = 7.1,
        z = 6.68
    ),
    carla.Rotation(
        pitch = -35,
        yaw = 127,
        roll = 0
    )
)

inter1pv5 = carla.Transform(
    carla.Location(
        x = -32,
        y = 32.5,
        z = 3.5
    ),
    carla.Rotation(
        pitch = -23,
        yaw = -103,
        roll = 0
    )
)

inter1pv6 = carla.Transform(
    carla.Location(
        x = -64.2,
        y = 7.7,
        z = 5
    ),
    carla.Rotation(
        pitch = -27,
        yaw = 40,
        roll = 0
    )
)

#Intersection2 (museum)
inter2tv = carla.Transform(
    carla.Location(
        x=90,
        y=20,
        z=60
    ),
    carla.Rotation(
        pitch=-90,
        yaw=0,
        roll=0
    )
)
inter2pv1 = carla.Transform(
    carla.Location(
        x=115,
        y=40,
        z=8.5
    ),
    carla.Rotation(
        pitch=-14,
        yaw=-124,
        roll=0
    )
)
inter2pv2 = carla.Transform(
    carla.Location(
        x = 96,
        y = 46,
        z = 7.9
    ),
    carla.Rotation(
        pitch = -29,
        yaw = -72,
        roll = 0
    )
)
inter2pv3 = carla.Transform(
    carla.Location(
        x = 92.29,
        y = 3.84,
        z = 5.2
    ),
    carla.Rotation(
        pitch = -8.8,
        yaw = 73.2,
        roll = 0
    )
)
inter2pv4 = carla.Transform(
    carla.Location(
        x = 114.54,
        y = -8.95,
        z = 6.9
    ),
    carla.Rotation(
        pitch = -25.81,
        yaw = 124.43,
        roll = 0
    )
)

spectator.set_transform(inter1tv)

### parameters and spawning

In [14]:
# Determine Camera Parameters
camera_bp = bp_lib.find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')

camera_bp_seg = bp_lib.find('sensor.camera.instance_segmentation')
camera_bp_seg.set_attribute('image_size_x', '1920')
camera_bp_seg.set_attribute('image_size_y', '1080')

## set view
c_pv_spawn_location = inter1pv1
c_tv_spawn_location = inter1tv

# Create a queue to store and retrieve the sensor data
camera_pv = world.spawn_actor(camera_bp, c_pv_spawn_location)
camera_pv_seg = world.spawn_actor(camera_bp_seg, c_pv_spawn_location)
camera_tv = world.spawn_actor(camera_bp, c_tv_spawn_location)


# Set up the simulator in synchronous mode
settings = world.get_settings()
settings.synchronous_mode = True 
settings.fixed_delta_seconds = 0.1
world.apply_settings(settings)

image_queue_tv = queue.Queue()
image_queue_pv = queue.Queue()
image_queue_pv_seg = queue.Queue()
camera_pv.listen(image_queue_pv.put)
camera_pv_seg.listen(image_queue_pv_seg.put)
camera_tv.listen(image_queue_tv.put)

actors = []

## Determine Camera Matrices to calculate World to Camera coordinates

In [15]:
def build_intrinsic_matrix(w, h, fov):
    focal = w / (2.0 * np.tan(fov * np.pi / 360.0))
    K = np.identity(3)
    K[0, 0] = K[1, 1] = focal
    K[0, 2] = w / 2.0
    K[1, 2] = h / 2.0
    return K

def get_image_point(loc, K, w2c):
    # Calculate 2D projection of 3D coordinate

    # Format the input coordinate (loc is a carla.Position object)
    point = np.array([loc.x, loc.y, loc.z, 1])
    # transform to camera coordinates
    point_camera = np.dot(w2c, point)

    # New we must change from UE4's coordinate system to an "standard"
    # (x, y ,z) -> (y, -z, x)
    # and we remove the fourth componebonent also
    point_camera = [point_camera[1], -point_camera[2], point_camera[0]]

    # now project 3D->2D using the camera matrix
    point_img = np.dot(K, point_camera)
    # normalize
    point_img[0] /= point_img[2]
    point_img[1] /= point_img[2]

    return int(point_img[0]), int(point_img[1])
    
    
# Get the world to camera matrix
world_2_camera = np.array(camera_pv.get_transform().get_inverse_matrix())
world_2_camera_bev = np.array(camera_tv.get_transform().get_inverse_matrix())

# Get the attributes from the camera
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()
fov = camera_bp.get_attribute("fov").as_float()

# Calculate the camera intrinsic matrix
K = build_intrinsic_matrix(image_w, image_h, fov)

## Spawn Actors

### Vehicles

In [16]:
for npc in world.get_actors().filter('*vehicle*'):
    npc.destroy()
    actors = []
    
# Spawn Vehicles
for i in range(100):
    vehicle_bp = random.choice(bp_lib.filter('vehicle'))
    if int(vehicle_bp.get_attribute('number_of_wheels')) != 2:
        npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))
    if npc:
        actors.append(npc)
        npc.set_autopilot(True)

### Peds (currently buggy)

In [17]:
""" walker_bp = bp_lib.filter("walker.pedestrian.*")
controller_bp = bp_lib.find("controller.ai.walker")

for npc in world.get_actors().filter('*pedestrian*'):
    npc.destroy()

for i in range(70):
    trans = carla.Transform()
    #trans.location = carla.Location(20, 20, 0)
    trans.location = world.get_random_location_from_navigation()
    trans.location.z += 1
    
    walker = random.choice(walker_bp)
    actor = world.try_spawn_actor(walker, trans)
    if actor:
        actors.append(actor)
    world.tick()
    controller = world.try_spawn_actor(controller_bp, carla.Transform(), actor)
    controller.start()    
    world.tick()
    controller.go_to_location(world.get_random_location_from_navigation()) """

' walker_bp = bp_lib.filter("walker.pedestrian.*")\ncontroller_bp = bp_lib.find("controller.ai.walker")\n\nfor npc in world.get_actors().filter(\'*pedestrian*\'):\n    npc.destroy()\n\nfor i in range(70):\n    trans = carla.Transform()\n    #trans.location = carla.Location(20, 20, 0)\n    trans.location = world.get_random_location_from_navigation()\n    trans.location.z += 1\n    \n    walker = random.choice(walker_bp)\n    actor = world.try_spawn_actor(walker, trans)\n    if actor:\n        actors.append(actor)\n    world.tick()\n    controller = world.try_spawn_actor(controller_bp, carla.Transform(), actor)\n    controller.start()    \n    world.tick()\n    controller.go_to_location(world.get_random_location_from_navigation()) '

## Preview

In [18]:
import math
def calculate_img_bb(bb):
    verts = [get_image_point(v, K, world_2_camera) for v in bb.get_world_vertices(npc.get_transform())]
    return verts

edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]] # For drawing BBOXES

dist_cutoff = 90

while True:
    world.tick()
    image_pv = image_queue_pv.get()
    image_tv = image_queue_tv.get()
    image_pv_seg = image_queue_pv_seg.get()
    
    # Reshape the raw data into an RGB array
    img_pv = np.reshape(np.copy(image_pv.raw_data), (image_pv.height, image_pv.width, 4)) 
    img_tv = np.reshape(np.copy(image_tv.raw_data), (image_tv.height, image_tv.width, 4)) 
    img_pv_seg = np.reshape(np.copy(image_pv_seg.raw_data), (image_pv_seg.height, image_pv_seg.width, 4)) 
    

    
    #for npc in world.get_actors().filter('*vehicle*').append(world.get_actors().filter('*pedestrian*')):
    for npc in actors:
        dist = npc.get_transform().location.distance(camera_pv.get_transform().location)
        bb = npc.bounding_box
        
        # Filter for the vehicles within 50m
        if dist > dist_cutoff:
            continue
            
        # Calculate the dot product between the forward vector
        # of the camera and the vector between the camera
        # and the npc. We threshold this dot product
        # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
        # TODO do this correct...
        forward_vec = camera_pv.get_transform().get_forward_vector() 
        ray = npc.get_transform().location - camera_pv.get_transform().location
        angle = math.acos(forward_vec.dot(ray) / (forward_vec.dot(forward_vec) * ray.dot(ray)))
        angle = math.degrees(angle)
        
        if forward_vec.dot(ray) > 1:    
            #print(angle) # Weird angle.
            #if angle > 45:
            #    continue
            
            # PV
            gcp_x, gcp_y = get_image_point(npc.get_location(), K, world_2_camera)
        
            fvec = npc.get_transform().get_forward_vector()
            psi = npc.get_location().x + fvec.x, npc.get_location().y + fvec.y, npc.get_location().z + fvec.z
            psi_x, psi_y = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera)

            cv2.circle(img_pv, (gcp_x, gcp_y), 3, (0, 100, 200), -1)
            cv2.arrowedLine(img_pv, (gcp_x, gcp_y), (psi_x, psi_y), (255, 0, 0, 255), 2)
            
            # Bev
            gcp_x_bev, gcp_y_bev = get_image_point(npc.get_location(), K, world_2_camera_bev)
            psi_x_bev, psi_y_bev = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera_bev)
            cv2.circle(img_tv, (gcp_x_bev, gcp_y_bev), 3, (0, 100, 200), -1)
            cv2.arrowedLine(img_tv, (gcp_x_bev, gcp_y_bev), (psi_x_bev, psi_y_bev), (255, 0, 0, 255), 2)
  
    
    # Display the image in an OpenCV display window
    cv2.namedWindow('Camera 1', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Camera 1', img_pv)
    cv2.namedWindow('Camera 1 Seg', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Camera 1 Seg', img_pv_seg)
    cv2.namedWindow('Top View', cv2.WINDOW_AUTOSIZE)
    cv2.imshow('Top View', img_tv)
    if cv2.waitKey(1) == ord('q'):
        break
cv2.destroyAllWindows()

## Make a Dataset

In [19]:
## variables for dataset generation
dist_cutoff = 50
ts = 0
iteration = 0
max_timesteps = 500

import pickle
import os
import glob
from tqdm import tqdm

def calculate_img_bb(bb):
    verts = [get_image_point(v, K, world_2_camera) for v in bb.get_world_vertices(npc.get_transform())]
    return verts

edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]] # For drawing BBOXES

# Delete Dataset
files = glob.glob('output/pv/*')
for f in files:
    os.remove(f)
files = glob.glob('output/tv/*')
for f in files:
    os.remove(f)
files = glob.glob('output/seg/*')
for f in files:
    os.remove(f)

vehicles_pv_per_ts = dict()
vehicles_tv_per_ts = dict()


pbar = tqdm(total = max_timesteps)
while True:
    world.tick()
    
    image_pv = image_queue_pv.get()
    image_tv = image_queue_tv.get()
    image_pv_seg = image_queue_pv_seg.get()
    iteration += 1
    
    frame_path_pv = 'output/pv/%06d' % image_pv.frame
    frame_path_tv = 'output/tv/%06d' % image_tv.frame
    frame_path_seg = 'output/seg/%06d' % image_pv_seg.frame
    
    # Save the image
    image_pv.save_to_disk(frame_path_pv + '.png')
    image_tv.save_to_disk(frame_path_tv + '.png')
    image_pv_seg.save_to_disk(frame_path_seg + '.png', color_converter=carla.ColorConverter.Raw)
    
      
    # Reshape the raw data into an RGB array
    img_pv = np.reshape(np.copy(image_pv.raw_data), (image_pv.height, image_pv.width, 4)) 
    img_tv = np.reshape(np.copy(image_tv.raw_data), (image_tv.height, image_tv.width, 4)) 
    img_pv_seg = np.reshape(np.copy(image_pv_seg.raw_data), (image_pv_seg.height, image_pv_seg.width, 4)) 
    
    vehicles_pv = []
    vehicles_tv = []
    
    for npc in world.get_actors().filter('*vehicle*'):
        dist = npc.get_transform().location.distance(camera_pv.get_transform().location)
        bb = npc.bounding_box
        
        # Filter for the vehicles within
        if dist > dist_cutoff:
            continue
            
        # Calculate the dot product between the forward vector
        # of the vehicle and the vector between the vehicle
        # and the other vehicle. We threshold this dot product
        # to limit to drawing bounding boxes IN FRONT OF THE CAMERA
        forward_vec = camera_pv.get_transform().get_forward_vector()
        ray = npc.get_transform().location - camera_pv.get_transform().location

        if forward_vec.dot(ray) > 1:
            bb_img = calculate_img_bb(bb)
            gcp_x, gcp_y = get_image_point(npc.get_location(), K, world_2_camera)
            fvec = npc.get_transform().get_forward_vector()
            psi = npc.get_location().x + fvec.x, npc.get_location().y + fvec.y, npc.get_location().z + fvec.z
            psi_x, psi_y = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera)
            gcp_x_bev, gcp_y_bev = get_image_point(npc.get_location(), K, world_2_camera_bev)
            psi_x_bev, psi_y_bev = get_image_point(carla.Location(psi[0], psi[1], psi[2]), K, world_2_camera_bev)
            
            entry_pv = {'id': npc.id, 'gcp': [gcp_x, gcp_y], 'psi': [psi_x, psi_y], 'img': frame_path_pv + '.png', 'bb': bb_img}
            entry_tv = {'id': npc.id, 'gcp': [gcp_x_bev, gcp_y_bev], 'psi': [psi_x_bev, psi_y_bev], 'img': frame_path_tv + '.png', 'bb': bb_img}
            vehicles_pv.append(entry_pv)
            vehicles_tv.append(entry_tv)
            
            #viz
            cv2.circle(img_pv, (gcp_x, gcp_y), 3, (0, 100, 200), -1)
            cv2.arrowedLine(img_pv, (gcp_x, gcp_y), (psi_x, psi_y), (255, 0, 0, 255), 2)
            cv2.circle(img_tv, (gcp_x_bev, gcp_y_bev), 3, (0, 100, 200), -1)
            cv2.arrowedLine(img_tv, (gcp_x_bev, gcp_y_bev), (psi_x_bev, psi_y_bev), (255, 0, 0, 255), 2)
    #print('iteration', iteration)
    pbar.update(1)
    vehicles_pv_per_ts[iteration] = vehicles_pv
    vehicles_tv_per_ts[iteration] = vehicles_tv
    #preview windows
    #cv2.namedWindow('Camera 1', cv2.WINDOW_AUTOSIZE)
    #cv2.imshow('Camera 1', img_pv)
    #cv2.namedWindow('Camera 1 Seg', cv2.WINDOW_AUTOSIZE)
    #cv2.imshow('Camera 1 Seg', img_pv_seg)
    #cv2.namedWindow('Top View', cv2.WINDOW_AUTOSIZE)
    #cv2.imshow('Top View', img_tv)
    if cv2.waitKey(1) == ord('q'):
        break

    if iteration >= max_timesteps:
        break

##ende
pbar.close()
cv2.destroyAllWindows()

#create pickle dumps
""" with open('output/pv/data.pickle', 'wb') as handle:
    pickle.dump(vehicles_pv_per_ts, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('output/tv/data.pickle', 'wb') as handle:
    pickle.dump(vehicles_tv_per_ts, handle, protocol=pickle.HIGHEST_PROTOCOL) """


 16%|█▋        | 82/500 [03:13<16:35,  2.38s/it]

## Package

### definitions

In [25]:
import random 
import pickle
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import numpy.typing as npt
from dataclasses import dataclass

@dataclass(frozen=True)
class Instance:
    id: int  # vehicle id from carla
    ts: int  # timestamp should also be the image name in seg, pv and tv folders
    # pv
    gcp_pv: npt.NDArray[np.int_]  # [2,] Ground contact point
    psi_pv: npt.NDArray[np.int_]  # [2,] point for the direction of the vehicle
    bb_pv: npt.NDArray[np.int_]   # [8, 2] # 3d bounding box of the vehicle
    hull_pv: npt.NDArray[np.int_]  # [n, 2] # hull of the vehicle
    image_pv: str        # path to the image
    # tv
    gcp_tv: npt.NDArray[np.int_]  # [2,] Ground contact point
    psi_tv: npt.NDArray[np.int_]  # [2,] point for the direction of the vehicle
    bb_tv: npt.NDArray[np.int_]   # [8, 2] # 3d bounding box of the vehicle
    image_tv: str        # path to the image

street_colors = []

def is_vehicle_out_of_bounds(gcp, w=1920, h=1080):
    x, y = gcp
    if x > w or x < 0 or y > h or y < 0:
        return True
    return False

def get_vehicles_from_ts(data, ts):
    return data[ts]

def get_vehicles_from_random_ts(data):
    idx_ts = random.choice(list(data.keys()))
    return idx_ts, data[idx_ts]

def filter_vehicles_not_in_img(data, w=1920, h=1080):
    return [vehicle for vehicle in data if not is_vehicle_out_of_bounds(vehicle['gcp'], w, h)]

def get_random_vehicle(data):
    idx = random.randint(0, len(data)-1)
    return data[idx]

def get_vehicles_from_ts(data, ts):
    return data[ts]

def get_vehicle_ids(data):
    return [vehicle['id'] for vehicle in data]

def filter_tv_vehicles(data, vehicle_ids):
    return [vehicle for vehicle in data if vehicle['id'] in vehicle_ids]

def cut_mask(bb, image):
    bb2d = bb_to_2d(np.array(bb))
    xmin, ymin, xmax, ymax = bb2d
    image_cropped = np.asarray(image)[ymin:ymax, xmin:xmax]
    return image_cropped

def get_mask(bb, image_cropped, image_seg):
    rgba, counts = np.unique(image_cropped.reshape(-1,4), axis=0, return_counts=True)
    # Filter street ((1, 51, 115, 255) inters2)
    mask = ~np.all((rgba == np.array([1, 94, 110, 255])), axis=1)

    rgba = rgba[mask]
    counts = counts[mask]
    target_value = rgba[np.argmax(counts)]
    mask = np.all(np.asarray(image_seg) == target_value, axis=-1)
    id = target_value[0]
    if (id == 1 or id == 2 or id == 3 or id == 9):
        street_colors.append(target_value)
    return mask

def get_hull(mask):
    object_pixels = np.column_stack(np.where(mask))
    hull = cv2.convexHull(object_pixels)
    hull = hull.squeeze()
    return hull

def bb_to_2d(bb):
    x_min, x_max = np.min(bb[:, 0]), np.max(bb[:, 0])
    y_min, y_max = np.min(bb[:, 1]), np.max(bb[:, 1])
    return x_min, y_min, x_max, y_max

def plot_2d_box(bb):
    x_min, y_min, x_max, y_max = bb_to_2d(bb)
    plt.plot([x_min, x_min], [y_min, y_max], color='red')
    plt.plot([x_max, x_max], [y_min, y_max], color='red')
    plt.plot([x_min, x_max], [y_min, y_min], color='red')
    plt.plot([x_min, x_max], [y_max, y_max], color='red')

def plot_3d_box(bb):
    edges = [[0,1], [1,3], [3,2], [2,0], [0,4], [4,5], [5,1], [5,7], [7,6], [6,4], [6,2], [7,3]]
    for edge in edges:
        plt.plot(bb[edge, 0], bb[edge, 1], color='blue')

def plot_gcp(gcp):
    _ = plt.scatter(gcp[0], gcp[1], color='cyan', marker='*', s=30)

def plot_psi(gcp, psi):
    x, y = gcp
    x2, y2 = psi
    _ = plt.plot([x, x2], [y, y2], color='red', lw=2)

def plot_hull(hull):
    hull = np.vstack((hull, hull[0, :]))
    plt.plot(hull[:, 1], hull[:, 0], color='orange')


def append_min_area_rect(data):
    for vehicle in data:
        min_area_rect = cv2.minAreaRect(vehicle['hull']) # Maybe need rotation
        box = cv2.boxPoints(min_area_rect)
        box = np.intp(box)
        vehicle['min_area_rect'] = box
    return data

def plot_min_area_rect(box):
    box = np.vstack((box, box[0, :]))
    plt.plot(box[:, 1], box[:, 0], color='cyan')

def get_data(ts, base_url='./', w=1980, h=1080):
    # Load Pickle Data
    with open(base_url + 'output/pv/data.pickle', 'rb') as handle:
        data_pv = pickle.load(handle)

    with open(base_url + 'output/tv/data.pickle', 'rb') as handle:
        data_tv = pickle.load(handle)
        
    # Perspective View
    vehicles_at_ts = get_vehicles_from_ts(data_pv, ts)
    vehicles_at_ts_filtered = filter_vehicles_not_in_img(vehicles_at_ts, w, h)
    vehicles_pv = append_hull(base_url, vehicles_at_ts_filtered)
    vehicles_pv = append_min_area_rect(vehicles_at_ts_filtered)
    fname_pv = base_url + vehicles_at_ts_filtered[0]['img']
    
    # Top View
    vehicles_at_ts_tv = get_vehicles_from_ts(data_tv, ts)
    v_ids = get_vehicle_ids(vehicles_pv)
    vehicles_tv = filter_tv_vehicles(vehicles_at_ts_tv, v_ids)
    fname_tv = base_url+vehicles_tv[0]['img']

def clean_segmask(image_seg):
    """ Removes all non-car pixels from the segmentation mask.
    Supposed to be called once per segmentation mask.
    Args:
    image_seg (_type_): Segmentation mask opened with opencv in rgba format
    Returns:
    _type_: Mask with only car pixels
    """
    cars_min = np.array([0, 0, 13, 0])
    cars_max = np.array([255, 255, 15, 255])
    mask = cv2.inRange(image_seg, cars_min, cars_max)
    return mask
 
 
def cut_mask(bb, cleaned_mask):
    """Sets all pixels outside of the bounding box to 0.
    Args:
        bb (_type_): bounding box of the current vehicle
        cleaned_mask (_type_): Mask with all other colors expcept vehicles removed
    Returns:
        _type_: _description_
    """
    bb2d = bb_to_2d(np.array(bb))
    xmin, ymin, xmax, ymax = bb2d
    to_ignore = np.ones(cleaned_mask.shape[:2], dtype=bool)
    to_ignore[ymin:ymax, xmin:xmax] = False
    cleaned_mask = cleaned_mask.copy()
    cleaned_mask[to_ignore] = 0
    return cleaned_mask

def append_hull(base_url, data):
    fname = base_url+data[0]['img']
    fname_seg = base_url + 'output/seg/' + fname.split('/')[-1]
    image_seg = cv2.imread(fname_seg, cv2.IMREAD_UNCHANGED)
    image_seg = clean_segmask(image_seg)
    for vehicle in data:
        mask = cut_mask(vehicle['bb'], image_seg)
        hull = get_hull(mask)


        vehicle['hull'] = hull
    return data

def calculate_hull(seg_mask, bounding_box):
    mask = cut_mask(bounding_box, seg_mask)
    return get_hull(mask)
    


### execution

In [26]:
base_url = './'
w = 1980
h = 1080

all_instances = []

for i in vehicles_pv_per_ts.keys():
    data_pv = vehicles_pv_per_ts[i]
    data_tv = vehicles_tv_per_ts[i]
    vehicles_pv = filter_vehicles_not_in_img(data_pv, w, h)
    #vehicles_pv = append_hull(base_url, vehicles_at_ts_filtered)
    #vehicles_pv = append_min_area_rect(vehicles_at_ts_filtered)
    fname_pv = base_url + vehicles_at_ts_filtered[0]['img']
    #image_pv = Image.open(fname_pv)
    
    # Top View
    vehicles_at_ts_tv = get_vehicles_from_ts(data_tv, ts)
    v_ids = get_vehicle_ids(vehicles_pv)
    vehicles_tv = filter_tv_vehicles(data_tv, v_ids)
    list[tuple[float, float]]
    fname_tv = base_url+vehicles_tv[0]['img']
    assert len(vehicles_pv) == len(vehicles_tv)

    fname = base_url+data_pv[0]['img']
    fname_seg = base_url + 'output/seg/' + fname.split('/')[-1]
    image_seg = cv2.imread(fname_seg, cv2.IMREAD_UNCHANGED)
    image_seg = clean_segmask(image_seg)

    for vehicle_pv, vehicle_tv in zip(vehicles_pv, vehicles_tv):

        try:
            hull = calculate_hull(image_seg, vehicle_pv["bb"])
        except:
            continue # skip all vehicles where the hull cant be calculated
           
        instance = Instance(vehicle_pv['id'], 
                            i,
                            np.array(vehicle_pv['gcp']),
                            np.array(vehicle_pv['psi']),
                            np.array(vehicle_pv['bb']),
                            np.array(np.array(hull)),
                            vehicle_pv['img'] ,
                            np.array(vehicle_tv['gcp']),
                            np.array(vehicle_tv['psi']),
                            np.array(vehicle_tv['bb']),
                            vehicle_tv['img']
                            )
        all_instances.append(instance)

with open('output/instances.pickle', 'wb') as handle:
    pickle.dump(all_instances, handle, protocol=pickle.HIGHEST_PROTOCOL)

print(f"Saved {len(all_instances)} instances to pickle")

Saved 19 instances to pickle
